In [1]:
import numpy as np 
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import datetime
import pandas_datareader.data as web
import pprint
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pickle
import quandl
import os
py.init_notebook_mode(connected=True)

In [2]:



    
api_key = 'enter your quandl api'
    
def get_quandl_data(quandl_id):
   
        #Download and cache Quandl dataseries
        cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
        try:
            f = open(cache_path, 'rb')
            df = pickle.load(f)   
            print('Loaded {} from cache'.format(quandl_id))
        
        except (OSError, IOError) as e:
            print('Downloading {} from Quandl'.format(quandl_id))
            df = quandl.get(quandl_id, returns="pandas")
            df.to_pickle(cache_path)
            print('Cached {} at {}'.format(quandl_id, cache_path))
          
    
        return df
    
                # Create backup
        if exists(cache_path):
            backupname = cache_path + '.bak'
            if exists(backupname):
                os.remove(backupname)
            os.rename(picklename, backupname)

        # Save
        with open(cachepath, "wb") as f:
            df.to_pickle(cache_path)
       
# Pull Kraken BTC price exchange data


btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD',)
     
btc_usd_price_kraken.tail()






Loaded BCHARTS/KRAKENUSD from cache


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2019-08-27,10364.4,10387.9,10033.0,10177.3,3401.115957,3.457165e+07,10164.795670
2019-08-28,10180.0,10297.7,9533.0,9716.1,6687.993853,6.604279e+07,9874.828626
2019-08-29,9714.9,9714.9,9307.8,9484.3,5907.220445,5.614802e+07,9504.981706
2019-08-30,9484.7,9708.0,9350.5,9578.0,4391.339141,4.191076e+07,9543.958262
2019-08-31,9577.9,9690.0,9400.0,9630.7,1939.586242,1.857071e+07,9574.570749


In [3]:
        # Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [4]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [5]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)



In [6]:
# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

btc_usd_datasets

,KRAKEN,COINBASE,BITSTAMP,ITBIT
Date,,,,
2011-09-13,NaN,NaN,5.929231,NaN
2011-09-14,NaN,NaN,5.590798,NaN
2011-09-15,NaN,NaN,5.094272,NaN
2011-09-16,NaN,NaN,4.854515,NaN
2011-09-17,NaN,NaN,4.870000,NaN
2011-09-18,NaN,NaN,4.839576,NaN
2011-09-19,NaN,NaN,4.900000,NaN
2011-09-20,NaN,NaN,5.388054,NaN
2011-09-21,NaN,NaN,5.706075,NaN


In [7]:
def df_scatter(df, title, separate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
    title=title,
    legend=dict(orientation="h"),
    xaxis=dict(type='date'),
    yaxis=dict(
        title=y_axis_label,
        showticklabels= not separate_y_axis,
        type=scale
    )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'True'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible = True
        )
        
        # Add separate axis for the series
        if separate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [8]:
# Remove "0" values
btc_usd_datasets.replace(0, np.nan, inplace=True)

In [9]:
# Plot the revised dataframe
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

In [10]:
# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)

# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_datasets.index, y=btc_usd_datasets['avg_btc_price_usd'])
py.iplot([btc_trace])

In [11]:
data = 'avg_btc_price_usd'

avg1 = btc_usd_datasets[data].rolling(360).mean() 
avg2= btc_usd_datasets[data].rolling(200).mean() 
avg3 = btc_usd_datasets[data].rolling(100).mean() 
avg4 = btc_usd_datasets[data].rolling(50).mean() 
avg5 = btc_usd_datasets[data].rolling(21).mean() 



start = '2018-6-20'





x= btc_usd_datasets.loc[start:]

x = x.index





#x = x['Date']



In [12]:
trace1 = go.Scatter( 
    
    
    x= x,
    y=btc_usd_datasets[data],
    
    name = data
    
)

trace2 = go.Scatter(
    x= x,
    y= avg1,
    mode='markers',
    marker=dict(
        size=2,
        color='cyan',
    ),
    name='MA360'
)

trace3 = go.Scatter(
    x= x,
    y= avg2,
    mode='markers',
    marker=dict(
        size=2,
        color='green',
    ),
    name='MA200'
)
    
trace4 = go.Scatter(
    x= x,
    y= avg3,
    mode='markers',
    marker=dict(
        size=2,
        color='red',
    ),
    name='MA100'
)    



trace5 = go.Scatter(
    x= x,
    y= avg4,
    mode='markers',
    marker=dict(
        size=2,
        color='purple',
    ),
    name='MA50'
)    

trace6 = go.Scatter(
    x= x,
    y= avg5,
    mode='markers',
    marker=dict(
        size=2,
        color='purple',
    ),
    name='MA21'
)    
layout = go.Layout(
    yaxis=dict(
        #type='scale',
        autorange=True
    )
)

data = [trace1, trace2, trace3, trace4, trace5,trace6]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [13]:
ret  = np.log(btc_usd_datasets['avg_btc_price_usd']/btc_usd_datasets['avg_btc_price_usd'].shift(1)).fillna(0)

In [14]:
ret.head()

Date
2011-09-13    0.000000
2011-09-14   -0.058773
2011-09-15   -0.093005
2011-09-16   -0.048208
2011-09-17    0.003185
Name: avg_btc_price_usd, dtype: float64

In [15]:
ret_trace = go.Scatter(x=btc_usd_datasets.index, y=ret)
py.iplot([ret_trace])

In [16]:
from statsmodels.tsa.statespace.kalman_filter import KalmanFilter

kf = KalmanFilter(1,1)

kf.obs_cov = np.array([.5]) # H
kf.state_cov = np.array([1.0])  # Q
kf.design = np.array([1.0])  # Z
kf.transition = np.array([1.0])  # T
kf.selection = np.array([1.0])  # R

ts = btc_usd_datasets['BITSTAMP']



obs_matrix = ts.values.copy()

# Initialize known state
kf.initialize_known(np.array(obs_matrix[0:1]), np.array([[1.0]]))
# Bind data

 

#print (obs_matrix[0:1])
kf.bind(obs_matrix.copy())
r = kf.filter()


In [17]:
a=len(obs_matrix)


kfilter1 = np.array (r.filtered_state[0][0:a])



btc_usd_datasets['filtered_state'] = kfilter1




btc_usd_datasets[['BITSTAMP','filtered_state']].plot(figsize=(8,10), style=['r-', 'b--',],yticks =np.arange(0, 140,10))







In [18]:
a= btc_usd_datasets[['BITSTAMP','filtered_state']]



df_scatter(a,' Kalman Forecast')



In [19]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm



model = pm.auto_arima(btc_usd_datasets[data], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True
                      #,out_of_sample_size=40
                     )

print(model.summary())


ModuleNotFoundError: No module named 'pmdarima'